In [28]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import csv
from torchvision import models, transforms

In [29]:
from torch.utils.data import Dataset
from PIL import Image, UnidentifiedImageError
import glob
import os
class MyDigitDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            image_paths.extend(glob.glob(os.path.join(root_dir, '*', 'hand_written_digit', '*', ext)))
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = int(os.path.basename(img_path).split('_')[0])
        if self.transform:
            image = self.transform(image)
        return image, label
class MyTestDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            for path in glob.glob(os.path.join(folder, ext)):
                try:
                    # Kiểm tra ảnh có mở được không
                    with Image.open(path) as img:
                        img.verify()  # không load nội dung, chỉ xác minh định dạng
                    self.image_paths.append(path)
                except (UnidentifiedImageError, OSError):
                    print(f"Bỏ ảnh lỗi: {path}")  # hoặc ghi log

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path


In [30]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [31]:
root_path = r'C:\Users\Admin\Desktop\python\doan'
traindata = MyDigitDataset(root_path, transform=transform)
dataloader = DataLoader(traindata, batch_size=64, shuffle=True, num_workers=0)

test_folder = r'C:\Users\Admin\Desktop\python\cs114'
testdata = MyTestDataset(test_folder, transform=transform)
test_loader = DataLoader(testdata, batch_size=64,num_workers=0, shuffle=False)

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=True) # true = trong số đã train, false = tự train
model.classifier[1] = nn.Linear(model.last_channel, 10)  # 10 chữ số (0–9)
for param in model.features.parameters():
    param.requires_grad = False
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [33]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)



In [34]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss:.4f} - Acc: {100 * correct / total:.2f}%")

Epoch [1/10] - Loss: 182.0538 - Acc: 36.59%
Epoch [2/10] - Loss: 136.7298 - Acc: 55.90%
Epoch [3/10] - Loss: 122.6661 - Acc: 60.70%
Epoch [4/10] - Loss: 115.7427 - Acc: 62.37%
Epoch [5/10] - Loss: 108.9859 - Acc: 64.08%
Epoch [6/10] - Loss: 105.0187 - Acc: 66.16%
Epoch [7/10] - Loss: 105.0894 - Acc: 65.36%
Epoch [8/10] - Loss: 100.8768 - Acc: 67.68%
Epoch [9/10] - Loss: 100.0220 - Acc: 67.04%
Epoch [10/10] - Loss: 97.9804 - Acc: 67.49%


In [35]:
model.eval()
results = []

with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()  # chuyển về numpy để dễ xử lý

        for path, pred in zip(img_paths, preds):
            filename = os.path.basename(path)
            results.append([filename, int(pred)])

# Ghi ra file CSV
with open('MOBILENETV2_NOT_TRAIN.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'prediction'])  # header
    writer.writerows(results)